***
# **Required Variable Settings** 

In [10]:
subreddit_here = "Ask_Politics"

pos_mean_pb_threshold = 0.6
neg_mean_pb_treshold = 0.4
var_threshold = 0.025

# define optional sample fraction 

sample_boolean = False
sample_fraction = 0.1

num_groups = 2

# adapt file name for csv saving
import_path = f"optimization_data_{subreddit_here}_frac1.csv"
output_path = f"result_optimization_data_{subreddit_here}_frac1.csv"

# Gurobi
options = {
    "WLSACCESSID" : "226ae632-51f7-4012-8eb5-455b1cb1803f",
    "WLSSECRET" : "fdef9059-60d1-42fb-b0e1-a16ba13efe03",
    "LICENSEID" : 2597393
 
}

In [ ]:
import sys
sys.path
sys.path.insert(1, '/home/scc/elena.solar')
sys.path.append('/home/scc/elena.solar/.local/lib/python3.10/site-packages')

***

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import matplotlib.pyplot as plt
import seaborn as sns
import community as community_louvain

from sklearn.model_selection import train_test_split
from scipy import sparse
import random
from itertools import combinations
import collections
import multiprocessing
import copy
import time
import gurobipy
from gurobipy import *

In [4]:
# # import self written functions
# from functions.script_faultana_functions import filter_subreddit
# from functions.script_faultana_functions import aggregate_interactions_directed
# from functions.script_faultana_functions import get_counts_and_shares
# from functions.script_faultana_functions import find_max
# from functions.script_faultana_functions import build_undirected


In [5]:
# import prepared data for optimization
net_sample = pd.read_csv(import_path)
net_sample

In [ ]:
user_sample = set(list(net_sample['user_1_id']) + list(net_sample['user_2_id']))

n_users = len(user_sample)

In [25]:
# for optimzation, drop OG ids
net_sample.drop(columns = {'user_1_id', 'user_2_id'}, inplace = True)


In [ ]:
n_users = len(user_sample)

***
## **Optimization**

In [31]:
Graph=[]
signedMatrix=[]
unsignedMatrix=[]
order = []
size= []
ShuffledEdges=[]
sorted_weighted_edges=[]
NumberOfNegative  = []

In [ ]:
    
# adapt network-df to be a matrix
matrix = sparse.csr_matrix((net_sample.sign.values, (net_sample.source.values, net_sample.target.values)),shape=(n_users, n_users))
arr = matrix.toarray()
arr

# should always be empty for me
print('Ambiguous edges') 
amb = np.where(((arr== (-1)*arr.T)&(arr!= 0)))
print(np.shape(amb)[1])

# save array, (directed) Graph and H (undirected Graph)
A = arr
G = nx.from_numpy_array(A, create_using=nx.MultiDiGraph()) # create directed network
H = G.to_undirected() # undirected


# append to dictionaries
Graph.append(nx.Graph(H))
matrix = nx.to_numpy_array(Graph[0])
order.append(len(np.matrix(matrix))) # 722 --> nr. of nodes per network
signedMatrix.append(matrix) 

# counts of interactions (independant of the type of interaction)
unsignedMatrix.append(abs(signedMatrix[0]))

# size = count of relations we have
size.append(int(np.count_nonzero(signedMatrix[0]))/2)

# sign (pos or negative) of an edge is stored in the weights
weighted_edges=nx.get_edge_attributes(Graph[0], 'weight') 

# sort the edges by their nodes id (smaller id, larger id)
sorted_weighted_edges.append({})
for (u,v) in weighted_edges:
    if u<v:
        (sorted_weighted_edges[0])[(u,v)] = weighted_edges[(u,v)]
    if u>v:
        (sorted_weighted_edges[0])[(v,u)] = weighted_edges[(u,v)]     


# do I have to change from /2 to keeping, because I already have unsigned relations?
NumberOfNegative.append(np.count_nonzero(signedMatrix[0] == -1)/2)


Ambiguous edges
0


In [ ]:
# check
net_sample
n_users
Graph
signedMatrix
unsignedMatrix 
order 
size 
ShuffledEdges 
sorted_weighted_edges
NumberOfNegative 

[4273.0]

In [37]:
# settings, because initially intended as loop
index = 0
net = len(user_sample) # n_users

In [ ]:
maximum_colour = num_groups # defined at beginning of notebook

objectivevalue=[]
solveTime=[]
effectiveBranchingFactors=[]
optimal_partition=[]



solutions = {}

## was intended as loop
#for index,net in enumerate(networks):


degree=(signedMatrix[index]).sum(1)
unsignedDegree=(unsignedMatrix[index]).sum(1)

x=[]
f={}


# Model
env = Env(params=options)
model = Model("Multi-colour frustration",env=env)

# How many threads to be used for exploring the feasible space in parallel?
# Here, the minimum of 32 and the availbale CPUs is used
model.setParam(GRB.Param.Threads, min(32,multiprocessing.cpu_count()))

# Do you want details of branching to be reported? (0=No, 1=Yes)
model.setParam(GRB.param.OutputFlag, 1) 

# For large MIPs and limited RAM, it is suggested to set this parameter for writing search tree nodes on disk
model.setParam("NodefileStart", 0.5)

# Do you want a non-zero Mixed integer programming tolerance (MIP Gap)?
# Note that a non-zero MIP gap may prevent the model from computing the exact value of frustration index
# model.setParam('MIPGap', 0.175)  

# There are different methods for solving optimization models:
# (-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent)
# For problems with a large number of contstraints, barrier method is more suitable
#model.setParam(GRB.param.Method, -1)

# Creating the decision variables
for i in range(0,order[index]):
    x.append([])
    for c in range(maximum_colour):
        x[i].append(model.addVar(lb=0.0, ub=1, vtype=GRB.BINARY, name='x'+str(i)+','+str(c))) # arguments by name

for (i,j) in (sorted_weighted_edges[index]):
        f[(i,j)]=model.addVar(lb=0.0, ub=1, vtype=GRB.BINARY, name='f'+str(i)+','+str(j))

# Updating the model to integrate new variables
model.update()

OFV=0        
for (i,j) in (sorted_weighted_edges[index]):
    OFV = OFV + f[(i,j)]              
model.setObjective(OFV, GRB.MINIMIZE)

# Adding constraints to the model
for c in range(maximum_colour):
    for (i,j) in (sorted_weighted_edges[index]):
        if (sorted_weighted_edges[index])[(i,j)]==1:
            model.addConstr( f[(i,j)] >= x[i][c] - x[j][c]  , 'posEdge'+','+str(i)+','+str(j))       
        if (sorted_weighted_edges[index])[(i,j)]==-1:
            model.addConstr( f[(i,j)] >= x[i][c] + x[j][c] -1  , 'negEdge'+','+str(i)+','+str(j))

# Constraint to ensure every node takes a color.
for i in range (0,order[index]):
    model.addConstr(gurobipy.quicksum((x[i][c] for c in range(maximum_colour))) == 1)
        
# Additional constraint to force all colors to be used
for c in range(maximum_colour):
    model.addConstr(gurobipy.quicksum((x[i][c] for i in range(order[index]))) >= 1)

#model.addConstr(OFV >= 200, 'LP lower bound')    
model.update()


# Solving the mathematical optimization model
start_time = time.time()
model.optimize()
solveTime.append(time.time() - start_time) 
print('Model', index,"solve time:",(time.time() - start_time))

obj = model.getObjective()
objectivevalue.append((obj.getValue()))
print('Model', index,' optimal solution equals',objectivevalue[index]) 

#when I use all time periods, use this:
print('Model', net,' optimal solution equals',objectivevalue[index]) 

# Constructing the optimal partition based on the optimal solution
optimal_partition.append([])
for v in model.getVars():
    if v.varName.startswith('x'):
        if v.x>0.9:
            (optimal_partition[index]).append(int(v.varName[-1]))
            #print (v.varName, v.x)   
#for color in optimal_partition[index]:
#    print(color)

df_save = pd.DataFrame(optimal_partition[index])
#df_save = df_save.T

df_save.rename(columns = {0 : 'Community'}, inplace = True)
df_save['Node'] = df_save.index

print(df_save)
solutions[net] = df_save
    
print("Clusterability indices ( k=",maximum_colour,"):",objectivevalue)
#print("frustrarion index Average",np.mean(objectivevalue))
#print("Frustration index SD     ",np.std(objectivevalue))
print("Solve times:",np.around(solveTime, decimals=2))
#print("Average solve time",np.mean(solveTime))
#print("Solve time Standard Deviation",np.std(solveTime))
print()
#for index in range(run):
#    print(objectivevalue[index])

df_indexes = pd.DataFrame(objectivevalue)
df_indexes.rename(columns = {0 : 'min_frus'}, inplace = True)
df_indexes.head()


df_save.to_csv(output_path, index = False)

Set parameter Username
Set parameter LicenseID to value 2574289
Academic license - for non-commercial use only - expires 2025-10-26
Set parameter Threads to value 8
Set parameter NodefileStart to value 0.5
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21981 rows, 12071 columns and 66658 nonzeros
Model fingerprint: 0x5e76f50a
Variable types: 0 continuous, 12071 integer (12071 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5366.0000000
Presolve removed 733 rows and 732 columns
Presolve time: 0.12s
Presolved: 21248 rows, 11339 columns, 63744 nonzeros
Found heuristic solution: objective 5340.0000000
Variab